In [8]:
import re
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers.core import Lambda
from keras.layers.merge import concatenate, add, multiply
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.noise import GaussianNoise
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
np.random.seed(0)
WNL = WordNetLemmatizer()
STOP_WORDS = set(stopwords.words('english'))
MAX_SEQUENCE_LENGTH = 30
MIN_WORD_OCCURRENCE = 100
REPLACE_WORD = "memento"
EMBEDDING_DIM = 300
NUM_FOLDS = 10
BATCH_SIZE = 1025
EMBEDDING_FILE = "../glove.840B.300d.txt"

In [4]:
def cutter(word):
    if len(word) < 4:
        return word
    return WNL.lemmatize(WNL.lemmatize(word, "n"), "v")

def preprocess(string):
    string = string.lower().replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'") \
        .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not") \
        .replace("n't", " not").replace("what's", "what is").replace("it's", "it is") \
        .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are") \
        .replace("he's", "he is").replace("she's", "she is").replace("'s", " own") \
        .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ") \
        .replace("€", " euro ").replace("'ll", " will").replace("=", " equal ").replace("+", " plus ")
    string = re.sub('[“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@]', ' ', string)
    string = re.sub(r"([0-9]+)000000", r"\1m", string)
    string = re.sub(r"([0-9]+)000", r"\1k", string)
    string = ' '.join([cutter(w) for w in string.split()])
    return string

def get_embedding():
    embeddings_index = {}
    f = open(EMBEDDING_FILE)
    for line in f:
        values = line.split()
        word = values[0]
        if len(values) == EMBEDDING_DIM + 1 and word in top_words:
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
    f.close()
    return embeddings_index

def is_numeric(s):
    return any(i.isdigit() for i in s)

def prepare(q):
    new_q = []      # New question after processing
    surplus_q = []  # set of words neither belong to [topwords, STOPWORDS, digits]
    numbers_q = []  # set of digits
    new_memento = True
    for w in q.split()[::-1]:  # Why in reverse order???
        if w in top_words:
            new_q = [w] + new_q
            new_memento = True
        elif w not in STOP_WORDS:
            if new_memento:
                new_q = ["memento"] + new_q
                new_memento = False
            if is_numeric(w):
                numbers_q = [w] + numbers_q
            else:
                surplus_q = [w] + surplus_q
        else:
            new_memento = True
        if len(new_q) == MAX_SEQUENCE_LENGTH:
            break
    new_q = " ".join(new_q)
    return new_q, set(surplus_q), set(numbers_q)

def extract_features(df):
    q1s = np.array([""] * len(df), dtype=object)
    q2s = np.array([""] * len(df), dtype=object)
    features = np.zeros((len(df), 4))

    for i, (q1, q2) in enumerate(list(zip(df["question1"], df["question2"]))):
        q1s[i], surplus1, numbers1 = prepare(q1)
        q2s[i], surplus2, numbers2 = prepare(q2)
        features[i, 0] = len(surplus1.intersection(surplus2))
        features[i, 1] = len(surplus1.union(surplus2))
        features[i, 2] = len(numbers1.intersection(numbers2))
        features[i, 3] = len(numbers1.union(numbers2))

    return q1s, q2s, features

In [5]:
dataset = 'quora'
data_dir, feature_dir = '../quora/', '../quora/'
os.mkdir('quora-models')
os.mkdir('quora-predictions')

# dataset = 'wiki'
# data_dir, feature_dir = '../wiki/', '../wiki/'
# os.mkdir('wiki-models')
# os.mkdir('wiki-predictions')

train = pd.read_csv(data_dir + "train.csv")

train["question1"] = train["question1"].fillna("").apply(preprocess)
train["question2"] = train["question2"].fillna("").apply(preprocess)

In [6]:
print("Creating the vocabulary of words occurred more than", MIN_WORD_OCCURRENCE)
all_questions = pd.Series(train["question1"].tolist() + train["question2"].tolist()).unique()
vectorizer = CountVectorizer(lowercase=False, token_pattern="\S+", min_df=MIN_WORD_OCCURRENCE)
vectorizer.fit(all_questions)
top_words = set(vectorizer.vocabulary_.keys())
top_words.add(REPLACE_WORD)

embeddings_index = get_embedding()
print("Words are not found in the embedding:", top_words - embeddings_index.keys())
top_words = embeddings_index.keys()

print("Train questions are being prepared for LSTM...")
q1s_train, q2s_train, train_q_features = extract_features(train)

tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(np.append(q1s_train, q2s_train))
word_index = tokenizer.word_index

data_1 = pad_sequences(tokenizer.texts_to_sequences(q1s_train), maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(tokenizer.texts_to_sequences(q2s_train), maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(train["is_duplicate"])

nb_words = len(word_index) + 1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print("Train features are being merged with NLP and Non-NLP features...")
train_nlp_features = pd.read_csv(feature_dir + "nlp_features_train.csv")
train_non_nlp_features = pd.read_csv(feature_dir + "non_nlp_features_train.csv")
features_train = np.hstack((train_q_features, train_nlp_features, train_non_nlp_features))

Creating the vocabulary of words occurred more than 100
Words are not found in the embedding: {'quorans', 'redmi', 'kvpy', 'oneplus', 'demonetisation', 'iisc', 'paytm', 'c#', '\\sqrt', '\\frac', 'brexit'}
Train questions are being prepared for LSTM...
Train features are being merged with NLP and Non-NLP features...


## Split 10% of train as valid to analyze performance

In [7]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, test_size=0.1, random_state=15)

In [10]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True)
model_count = 0

for idx_train, idx_val in skf.split(train["is_duplicate"], train["is_duplicate"]):
    print("MODEL:", model_count)
    data_1_train = data_1[idx_train]
    data_2_train = data_2[idx_train]
    labels_train = labels[idx_train]
    f_train = features_train[idx_train]

    data_1_val = data_1[idx_val]
    data_2_val = data_2[idx_val]
    labels_val = labels[idx_val]
    f_val = features_train[idx_val]

    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
    lstm_layer = LSTM(75, recurrent_dropout=0.2)

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = lstm_layer(embedded_sequences_1)

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = lstm_layer(embedded_sequences_2)

    features_input = Input(shape=(f_train.shape[1],), dtype="float32")
    features_dense = BatchNormalization()(features_input)
    features_dense = Dense(200, activation="relu")(features_dense)
    features_dense = Dropout(0.2)(features_dense)

    addition = add([x1, y1])
    minus_y1 = Lambda(lambda x: -x)(y1)
    merged = add([x1, minus_y1])
    merged = multiply([merged, merged])
    merged = concatenate([merged, addition])
    merged = Dropout(0.4)(merged)

    merged = concatenate([merged, features_dense])
    merged = BatchNormalization()(merged)
    merged = GaussianNoise(0.1)(merged)

    merged = Dense(150, activation="relu")(merged)
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged)

    out = Dense(1, activation="sigmoid")(merged)

    model = Model(inputs=[sequence_1_input, sequence_2_input, features_input], outputs=out)
    model.compile(loss="binary_crossentropy",
                  optimizer="nadam")
    early_stopping = EarlyStopping(monitor="val_loss", patience=5)
    
    best_model_path = os.path.join(dataset + '-models', "best_model" + str(model_count) + ".h5")
    model_checkpoint = ModelCheckpoint(best_model_path, save_best_only=True, save_weights_only=True)

    hist = model.fit([data_1_train, data_2_train, f_train], labels_train,
                     validation_data=([data_1_val, data_2_val, f_val], labels_val),
                     epochs=20, batch_size=BATCH_SIZE, shuffle=True,
                     callbacks=[early_stopping, model_checkpoint], verbose=2)

    model.load_weights(best_model_path)
    print(model_count, "validation loss:", min(hist.history["val_loss"]))
    
    # Predictions on leaved validation data # 
    leaved_valid_data_1 = data_1[valid.index]
    leaved_valid_data_2 = data_2[valid.index]
    f_leaved_valid = features_train[valid.index]
    
    preds = model.predict([leaved_valid_data_1, leaved_valid_data_2, f_leaved_valid], batch_size=BATCH_SIZE, verbose=1)

    submission = pd.DataFrame({"id": valid["id"], "is_duplicate": preds.ravel()})
    submission.to_csv(os.path.join(dataset + "-predictions", "preds" + str(model_count) + ".csv"), index=False)

    model_count += 1

MODEL: 0
Train on 327474 samples, validate on 36387 samples
Epoch 1/20
 - 55s - loss: 0.2716 - val_loss: 0.2361
Epoch 2/20
 - 52s - loss: 0.2375 - val_loss: 0.2303
Epoch 3/20
 - 52s - loss: 0.2251 - val_loss: 0.2126
Epoch 4/20
 - 52s - loss: 0.2160 - val_loss: 0.2083
Epoch 5/20
 - 52s - loss: 0.2090 - val_loss: 0.2053
Epoch 6/20
 - 52s - loss: 0.2032 - val_loss: 0.2112
Epoch 7/20
 - 52s - loss: 0.1980 - val_loss: 0.2020
Epoch 8/20
 - 52s - loss: 0.1937 - val_loss: 0.2005
Epoch 9/20
 - 52s - loss: 0.1891 - val_loss: 0.2013
Epoch 10/20
 - 52s - loss: 0.1850 - val_loss: 0.2000
Epoch 11/20
 - 52s - loss: 0.1819 - val_loss: 0.2003
Epoch 12/20
 - 52s - loss: 0.1790 - val_loss: 0.2002
Epoch 13/20
 - 52s - loss: 0.1760 - val_loss: 0.1996
Epoch 14/20
 - 52s - loss: 0.1737 - val_loss: 0.2008
Epoch 15/20
 - 52s - loss: 0.1707 - val_loss: 0.2019
Epoch 16/20
 - 52s - loss: 0.1679 - val_loss: 0.2024
Epoch 17/20
 - 52s - loss: 0.1663 - val_loss: 0.2047
Epoch 18/20
 - 52s - loss: 0.1651 - val_loss: 0.

Epoch 10/20
 - 52s - loss: 0.1850 - val_loss: 0.2040
Epoch 11/20
 - 52s - loss: 0.1814 - val_loss: 0.2076
Epoch 12/20
 - 52s - loss: 0.1785 - val_loss: 0.2080
Epoch 13/20
 - 52s - loss: 0.1748 - val_loss: 0.2085
Epoch 14/20
 - 52s - loss: 0.1727 - val_loss: 0.2071
Epoch 15/20
 - 52s - loss: 0.1702 - val_loss: 0.2059
7 validation loss: 0.2040382142277132
40429/40429 [==============================] - 3s 83us/step
MODEL: 8
Train on 327476 samples, validate on 36385 samples
Epoch 1/20
 - 57s - loss: 0.2743 - val_loss: 0.2472
Epoch 2/20
 - 52s - loss: 0.2386 - val_loss: 0.2197
Epoch 3/20
 - 52s - loss: 0.2253 - val_loss: 0.2136
Epoch 4/20
 - 52s - loss: 0.2171 - val_loss: 0.2084
Epoch 5/20
 - 52s - loss: 0.2101 - val_loss: 0.2071
Epoch 6/20
 - 52s - loss: 0.2046 - val_loss: 0.2069
Epoch 7/20
 - 52s - loss: 0.1992 - val_loss: 0.2010
Epoch 8/20
 - 52s - loss: 0.1950 - val_loss: 0.2006
Epoch 9/20
 - 52s - loss: 0.1898 - val_loss: 0.2001
Epoch 10/20
 - 52s - loss: 0.1862 - val_loss: 0.1992
Epo